In [1]:
import pandas as pd
from pymongo import MongoClient
import json, requests
import os
from dotenv import load_dotenv
load_dotenv()
import requests
from pandas.io.json import json_normalize

### Let's define a function that gets the nearest result from the Foursquare Api looking from the coordinates of the design office of 99Design in California. 

In [27]:
def getFoursquare(search):    
    url = 'https://api.foursquare.com/v2/venues/explore'
    params = dict(
    client_id=os.getenv('CLIENT_ID'),
    client_secret=os.getenv('CLIENT_SECRET'),
    v='20180323',
    ll='37.795531,-122.400598',
    query=search,
    limit=1
    )
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    df_api = pd.DataFrame([{
        'query' : data['response']['query'],
        'name': data['response']['groups'][0]['items'][0]['venue']['name'],
        'distance' :data['response']['groups'][0]['items'][0]['venue']['location']['distance'],
        'address': data['response']['groups'][0]['items'][0]['venue']['location']['formattedAddress'][0],
        'postal_code':data['response']['groups'][0]['items'][0]['venue']['location']['postalCode'],
        'latitude' : data['response']['suggestedBounds']['ne']['lat'],
        'longitude' : data['response']['suggestedBounds']['ne']['lng'] }])
    return df_api

### From the requirements we see that the most important one (the one that affect every person in the company) is the place to party. So i am going to launch a query search looking for the nearest pub and bar and find a midpoint in between both of them and use that as the default coordinates so that the office is right next to the pub and bar therefore giving the people two places to party

In [28]:
df1 = getFoursquare("pub")

In [29]:
df1

,query,name,distance,address,postal_code,latitude,longitude
0,pub,Old Ship Saloon,250,298 Pacific Ave (at Battery St),94111,37.799123,-122.399518


In [30]:
df2 = getFoursquare("bar")

In [31]:
df2

,query,name,distance,address,postal_code,latitude,longitude
0,bar,Roka | Bar,296,801 Montgomery St (at Jackson St),94133,37.797683,-122.402462


From this we see the cordinates of the bar and pub. Let's find the midpoint

In [33]:
#For the pub
lat1= 37.799123
lon1= -122.399518

In [34]:
#For the bar
lat2= 37.797683	
lon2= -122.402462

In [37]:
lat_final = (lat1+lat2)/2
lat_final

37.798403

In [39]:
lon_final= (lon1+lon2)/2
lon_final

-122.40099000000001

### Now we redefine the first function with our office coordinates which are the ones above

In [45]:
def getFoursquareFinal(search):    
    url = 'https://api.foursquare.com/v2/venues/explore'
    params = dict(
    client_id=os.getenv('CLIENT_ID'),
    client_secret=os.getenv('CLIENT_SECRET'),
    v='20180323',
    ll='37.798403,-122.400990',
    query=search,
    limit=1
    )
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    df_api = pd.DataFrame([{
        'query' : data['response']['query'],
        'name': data['response']['groups'][0]['items'][0]['venue']['name'],
        'distance' :data['response']['groups'][0]['items'][0]['venue']['location']['distance'],
        'address': data['response']['groups'][0]['items'][0]['venue']['location']['formattedAddress'][0],
        'postal_code':data['response']['groups'][0]['items'][0]['venue']['location']['postalCode'],
        'latitude' : data['response']['suggestedBounds']['ne']['lat'],
        'longitude' : data['response']['suggestedBounds']['ne']['lng'] }])
    return df_api

### Now let's look for the basketball stadium, which should be where the Golden State Warrios play, the Chase Center

In [46]:
df3 = getFoursquareFinal("basketball stadium")

In [47]:
df3

,query,name,distance,address,postal_code,latitude,longitude
0,basketball stadium,Chase Center,3601,1 Warriors Way,94158,37.769214,-122.38611


### Now let's find the closest starbucks

In [48]:
df4 = getFoursquareFinal("starbucks")

In [49]:
df4

,query,name,distance,address,postal_code,latitude,longitude
0,starbucks,Starbucks,471,398 Market St (Front St),94111,37.795749,-122.397875


### Now let's find an airport

In [50]:
df5 = getFoursquareFinal("airport")

In [51]:
df5

,query,name,distance,address,postal_code,latitude,longitude
0,airport,San Francisco International Airport (SFO) (San...,20262,N McDonnell Rd (at Link Rd),94128,37.618063,-122.38574


In [52]:
frames = [df1, df2, df3, df4, df5]
final = pd.concat(frames)
final

,query,name,distance,address,postal_code,latitude,longitude
0,pub,Old Ship Saloon,250,298 Pacific Ave (at Battery St),94111,37.799123,-122.399518
0,bar,Roka | Bar,296,801 Montgomery St (at Jackson St),94133,37.797683,-122.402462
0,basketball stadium,Chase Center,3601,1 Warriors Way,94158,37.769214,-122.386110
0,starbucks,Starbucks,471,398 Market St (Front St),94111,37.795749,-122.397875
0,airport,San Francisco International Airport (SFO) (San...,20262,N McDonnell Rd (at Link Rd),94128,37.618063,-122.385740


In [53]:
from folium import Map, Marker, Icon, FeatureGroup, LayerControl, Choropleth

In [60]:
m = Map(location=[37.798403,-122.400990],zoom_start=10)
for i, row in final.iterrows():
    icon = Icon(
                color="red",
                prefix="fa",
                icon="flag"
            )
    Marker(
        location=row[["latitude","longitude"]],
        tooltip=row["query"],
        popup=row["name"],
        icon=icon
    ).add_to(m)
m